In [ ]:
import consul
import sys
import uuid

In [47]:
SERVICE_NAME = sys.argv[0][:-3]
SERVICE_ID = f"{SERVICE_NAME}-{str(uuid.uuid4())[:4]}"

CONSUL_IP = "localhost"
CONSUL_PORT = 8500

SERVICE_IP = "host.docker.internal"
SERVICE_PORT = 5000

In [53]:
c = consul.Consul(host=CONSUL_IP, port=CONSUL_PORT)
try:
    leader = c.status.leader()
    print(f"Consul is available. Leader: {leader}")
except Exception as e:
    print(f"Failed to connect to Consul: {e}")

Consul is available. Leader: 172.24.0.2:8300


In [ ]:
c.agent.service.register(
    name=SERVICE_NAME,
    service_id=SERVICE_ID,
    address=SERVICE_IP,
    port=SERVICE_PORT,
    check=consul.Check.http(
        url=f"http://{SERVICE_IP}:{SERVICE_PORT}/health",
        interval="10s",
        timeout="1s",
        deregister="10m"
    )
)

True

### Get all services

In [65]:
services = c.agent.services()

for service_id, service_info in services.items():
    print(f"Service ID: {service_id}")
    print(f"Service Name: {service_info['Service']}")
    print(f"Service Address (IP): {service_info['Address']}")
    print("-" * 30)

Service ID: /home/stiro/.local/lib/python3.10/site-packages/ipykernel_launcher-9a5e
Service Name: /home/stiro/.local/lib/python3.10/site-packages/ipykernel_launcher
Service Address (IP): host.docker.internal
------------------------------
Service ID: server-hello-3507
Service Name: server-hello
Service Address (IP): 172.24.0.3
------------------------------


### Set a key

In [ ]:
# Set a key
key = 'a'
value = 'b'

c.kv.put(key, value)

print(f"Key '{key}' set successfully!")

### Get a key

In [ ]:
index, data = c.kv.get('a')

if data is not None:
    value = data['Value'].decode('utf-8')  # Must decode from bytes
    print(f"Key value: {value}")
else:
    print("Key not found.")
